In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
cable = pd.read_csv('./Data/training.csv', na_values=(-999, 6)) # value = 6 corresponds to refusal to answer, 6 nowhere else in data

In [ ]:
def CleanCableData(df):
    
    drop = ['YES', 'ID', 'age', 'class', 'tele_have']
    df['value'] = [(i - 3) for i in df['value']] # Normalize (-2 to +2)
    df = df[[col for col in df.columns if col not in drop]]
    df = df.dropna()
    
    return df

In [ ]:
cable = CleanCableData(cable)

In [ ]:
y = pd.DataFrame(cable['buy'])
X = cable[[col for col in cable.columns if col != 'buy']]

In [ ]:
Model = LogisticRegression()
Model.fit(X, y)

In [ ]:
def getSimulationRange(x, dx):
    
    # Accepts a series and a step: returns an ordered list ranging from the minimum to the maximum of the list
    # in the series, seperated by steps
    
    min_x = min(x)
    max_x = max(x)
    
    r = np.arange(min_x, max_x + dx, dx)
    
    return r

In [ ]:
def GenerateInputs(row, simulation):

    X = [np.insert(row, 16, i) for i in simulation]
    
    return X

In [ ]:
simulation = ['price'] # variables we have strong priors about
priors = np.array([]) # k x r?  this times a gradient should return a vector of booleans

sims = X[[col for col in simulation]] # extract features we wish to simulate

problem_space = X[[col for col in X.columns if col not in sims]] # all input vectors in sample
problem_space = problem_space.drop_duplicates().reset_index(drop=True) # unique input vectors in sample

In [ ]:
dx = 0.1

price_sim = getSimulationRange(sims['price'], dx)
price_sim # Exhaustive range of prices for simulation

In [ ]:
# This needs some work and does not hold in the multivariate case

for i, row in problem_space.iterrows():
    
    partials = []
    row = np.array(row)
    X_i = GenerateInputs(row, price_sim)
    
    y_i = np.array([p[1] for p in Model.predict_proba(X_i)])
    dy_dp = [((item - y_i[i]) / dx) for i, item in enumerate(y_i[1:])]
    
    partials = partials.append(dy_dp)
    #print(y_i)
    
    # f(y_i) = gradients
    
    # i maps back to neighborhoods in problem space
    
problem_space['dy/dp'] = dy_dp

# maybe: [gradient_f(Model.predict_proba(M)) for M in itertools.product(?)]

In [ ]:
dy_dp = [((item - y_i[i]) / dx) for i, item in enumerate(y_i[1:])]

In [ ]:
X_i_df = pd.DataFrame(X_i, columns=X.columns)